In [1]:
import numpy as np

In [42]:
A = np.array([[2, 6, 2],
    [3, 7, 1],
    [4, 5, 3]
], dtype=float)
A

array([[2., 6., 2.],
       [3., 7., 1.],
       [4., 5., 3.]])

In [43]:
np.linalg.inv(A)

array([[-0.667,  0.333,  0.333],
       [ 0.208,  0.083, -0.167],
       [ 0.542, -0.583,  0.167]])

In [44]:
np.eye(A.shape[0])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [45]:
n = A.shape[0]
aug = np.hstack((A.astype(float), np.eye(n)))
aug

array([[2., 6., 2., 1., 0., 0.],
       [3., 7., 1., 0., 1., 0.],
       [4., 5., 3., 0., 0., 1.]])

In [46]:
abs(aug[0:, 0]), abs(aug[1:, 1]), abs(aug[2:, 2])

(array([2., 3., 4.]), array([7., 5.]), array([3.]))

In [47]:
np.argmax(abs(aug[0:, 0])), np.argmax(abs(aug[1:, 1])) + 1, np.argmax(abs(aug[2:, 2])) + 2

(np.int64(2), np.int64(1), np.int64(2))

In [48]:
aug[[0, 2]], aug[[2, 0]] # switch them

(array([[2., 6., 2., 1., 0., 0.],
        [4., 5., 3., 0., 0., 1.]]),
 array([[4., 5., 3., 0., 0., 1.],
        [2., 6., 2., 1., 0., 0.]]))

In [49]:
aug[[0, 2]] = aug[[2, 0]]
aug

array([[4., 5., 3., 0., 0., 1.],
       [3., 7., 1., 0., 1., 0.],
       [2., 6., 2., 1., 0., 0.]])

In [50]:
pivot = aug[0, 0]
aug[0] = aug[0] / pivot
aug

array([[1.  , 1.25, 0.75, 0.  , 0.  , 0.25],
       [3.  , 7.  , 1.  , 0.  , 1.  , 0.  ],
       [2.  , 6.  , 2.  , 1.  , 0.  , 0.  ]])

In [51]:
for j in range(1, 3):
    factor = aug[j, 0]
    aug[j] -= factor * aug[0]

aug

array([[ 1.  ,  1.25,  0.75,  0.  ,  0.  ,  0.25],
       [ 0.  ,  3.25, -1.25,  0.  ,  1.  , -0.75],
       [ 0.  ,  3.5 ,  0.5 ,  1.  ,  0.  , -0.5 ]])

In [52]:
pivot = aug[1, 1]
aug[1] = aug[1] / pivot

np.set_printoptions(precision=3, suppress=True)
aug

array([[ 1.   ,  1.25 ,  0.75 ,  0.   ,  0.   ,  0.25 ],
       [ 0.   ,  1.   , -0.385,  0.   ,  0.308, -0.231],
       [ 0.   ,  3.5  ,  0.5  ,  1.   ,  0.   , -0.5  ]])

In [53]:
for j in range(2, 3):
    factor = aug[j, 1]
    aug[j] -= factor * aug[1]

aug

array([[ 1.   ,  1.25 ,  0.75 ,  0.   ,  0.   ,  0.25 ],
       [ 0.   ,  1.   , -0.385,  0.   ,  0.308, -0.231],
       [ 0.   ,  0.   ,  1.846,  1.   , -1.077,  0.308]])

In [54]:
pivot = aug[2, 2]
aug[2] = aug[2] / pivot

np.set_printoptions(precision=3, suppress=True)
aug

array([[ 1.   ,  1.25 ,  0.75 ,  0.   ,  0.   ,  0.25 ],
       [ 0.   ,  1.   , -0.385,  0.   ,  0.308, -0.231],
       [ 0.   ,  0.   ,  1.   ,  0.542, -0.583,  0.167]])

In [56]:
for i in range(n - 1, 0, -1):
    for j in range(i - 1, -1, -1):
        factor = aug[j][i]
        aug[j] -= factor * aug[i] 

aug

array([[ 1.   ,  0.   ,  0.   , -0.667,  0.333,  0.333],
       [ 0.   ,  1.   ,  0.   ,  0.208,  0.083, -0.167],
       [ 0.   ,  0.   ,  1.   ,  0.542, -0.583,  0.167]])

In [57]:
aug[:, n:]

array([[-0.667,  0.333,  0.333],
       [ 0.208,  0.083, -0.167],
       [ 0.542, -0.583,  0.167]])

In [64]:
np.set_printoptions(precision=20, suppress=True)
aug[:, n:] - np.linalg.inv(A)

array([[-0.00000000000000011102,  0.                    ,
         0.00000000000000011102],
       [ 0.                    , -0.00000000000000001388,
        -0.00000000000000002776],
       [ 0.                    ,  0.                    ,
        -0.00000000000000005551]])

In [65]:
def gaussian_inverse(A):
    """
    Invert matrix A using Gaussian elimination with partial pivoting.
    """
    n = A.shape[0]
    # Create augmented matrix [A | I]
    aug = np.hstack((A.astype(float), np.eye(n)))
    
    # Forward elimination
    for i in range(n):
        # Partial pivoting
        max_row = np.argmax(abs(aug[i:, i])) + i
        if aug[max_row, i] == 0:
            raise np.linalg.LinAlgError("Matrix is singular.")
        # Swap rows
        if max_row != i:
            aug[[i, max_row]] = aug[[max_row, i]]
        
        # Normalize pivot row
        pivot = aug[i, i]
        aug[i] = aug[i] / pivot
        
        # Eliminate below
        for j in range(i + 1, n):
            factor = aug[j, i]
            aug[j] -= factor * aug[i]
    
    # Back substitution
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            factor = aug[j, i]
            aug[j] -= factor * aug[i]
    
    # Extract inverse from augmented matrix
    return aug[:, n:]

In [66]:
inv_gauss = gaussian_inverse(A)
inv_gauss

array([[-0.6666666666666666 ,  0.33333333333333337,  0.33333333333333326],
       [ 0.20833333333333331,  0.08333333333333333, -0.16666666666666666],
       [ 0.5416666666666666 , -0.5833333333333334 ,  0.1666666666666667 ]])

Now we implement LU decomposition

In [68]:
def lu_decompose(A):
    """
    Perform LU decomposition with partial pivoting.
    Returns P, L, U such that P@A = L@U.
    """
    n = A.shape[0]
    U = A.copy().astype(float)
    L = np.eye(n)
    P = np.eye(n)
    
    for i in range(n):
        # Partial pivoting
        max_row = np.argmax(abs(U[i:, i])) + i
        if U[max_row, i] == 0:
            raise np.linalg.LinAlgError("Matrix is singular.")
        # Swap rows in U and P, and L (columns before i)
        if max_row != i:
            U[[i, max_row]] = U[[max_row, i]]
            P[[i, max_row]] = P[[max_row, i]]
            if i > 0:
                L[[i, max_row], :i] = L[[max_row, i], :i]
        
        # Eliminate below
        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j] -= L[j, i] * U[i]
    
    return P, L, U

In [72]:
np.set_printoptions(precision=3, suppress=True)
n = A.shape[0]
P, L, U = lu_decompose(A)
P, L, U

(array([[0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.]]),
 array([[1.   , 0.   , 0.   ],
        [0.5  , 1.   , 0.   ],
        [0.75 , 0.929, 1.   ]]),
 array([[ 4.   ,  5.   ,  3.   ],
        [ 0.   ,  3.5  ,  0.5  ],
        [ 0.   ,  0.   , -1.714]]))

In [74]:
inv = np.zeros_like(A, dtype=float)
I = np.eye(n)
P @ I[:, 0]

array([0., 1., 0.])

In [76]:
y = np.zeros(n)
L

array([[1.   , 0.   , 0.   ],
       [0.5  , 1.   , 0.   ],
       [0.75 , 0.929, 1.   ]])

In [79]:
def lu_inverse(A):
    """
    Invert matrix A using LU decomposition.
    """
    n = A.shape[0]
    P, L, U = lu_decompose(A)
    inv = np.zeros_like(A, dtype=float)
    
    # Solve for each column of the inverse
    I = np.eye(n)
    for i in range(n):
        # Solve Ly = P@e_i
        b = P @ I[:, i]
        y = np.zeros(n)
        for j in range(n):
            y[j] = b[j] - L[j, :j] @ y[:j]
        # Solve Ux = y
        x = np.zeros(n)
        for j in range(n - 1, -1, -1):
            x[j] = (y[j] - U[j, j+1:] @ x[j+1:]) / U[j, j]
        inv[:, i] = x
    return inv

In [80]:
inv_lu = lu_inverse(A)
inv_np = np.linalg.inv(A)
inv_np

array([[-0.667,  0.333,  0.333],
       [ 0.208,  0.083, -0.167],
       [ 0.542, -0.583,  0.167]])